In [36]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import explode, flatten, col, concat, lit, regexp_replace, input_file_name

In [37]:
spark = SparkSession.builder.master('local[*]') \
    .appName('visualizacao') \
    .getOrCreate()

In [38]:
df = spark.read.json('data/datalake/bronze/extract_date=*/*.json')
df.printSchema()
df = df.na.drop()

root
 |-- data_extracao: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



In [39]:
df_p = df.select(df.data_extracao, df.hr, explode(df.l).alias('lista'))
df_p.show(5, truncate=False)
df_p.printSchema()


+----------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|data_extracao   |hr   |lista                                                                                                                                                                                                                                                                                       

In [40]:
df_p2 = df_p.select(df_p.data_extracao, df_p.lista)
df_p2.printSchema()


root
 |-- data_extracao: string (nullable = true)
 |-- lista: struct (nullable = true)
 |    |-- c: string (nullable = true)
 |    |-- cl: long (nullable = true)
 |    |-- lt0: string (nullable = true)
 |    |-- lt1: string (nullable = true)
 |    |-- qv: long (nullable = true)
 |    |-- sl: long (nullable = true)
 |    |-- vs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |-- is: string (nullable = true)
 |    |    |    |-- p: long (nullable = true)
 |    |    |    |-- px: double (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- sv: string (nullable = true)
 |    |    |    |-- ta: string (nullable = true)



In [41]:
# df_amostra_um = df_p2.select(df_p2.data_extracao,df_p2.lista.c ,df_p2.lista.vs.p,df_p2.lista.vs.py, df_p2.lista.vs.px)
# Método para obter o dados dos ônibus
df_amostra_um = df_p2.select(
    df_p2.data_extracao,
    df_p2.lista.c.alias('LETREIRO_COMPLETO'), 
    df_p2.lista.sl.alias('SENTIDO_OPERACAO'), 
    df_p2.lista.cl.alias('CODIGO_IDENTIFICADOR_LINHA') , 
    explode(df_p2.lista.vs).alias('expansao')) \
    .select(col('data_extracao').alias('DATA_EXTRACAO'),
            'LETREIRO_COMPLETO',
            'SENTIDO_OPERACAO',
            'CODIGO_IDENTIFICADOR_LINHA',
            col('expansao.p').alias('PREFIXO_ONIBUS'),
            col('expansao.ta').alias('DATA_HORA_CAPTURA_LOCALIZACAO'),
            col('expansao.py').alias('LATITUDE') ,
            col('expansao.px').alias('LONGITUDE'),
            
            ) \
            .filter(col('expansao.p') == '15208') \
    .sort(df_p2.data_extracao.desc(),df_p2.lista.c.asc() ) \
    
df_amostra_um.show(truncate=False)


+----------------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|DATA_EXTRACAO   |LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |
+----------------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|2023-09-09 22:40|848L-10          |2               |33576                     |15208         |2023-09-10T01:39:54Z         |-23.4130405        |-46.750367         |
|2023-09-09 22:36|848L-10          |2               |33576                     |15208         |2023-09-10T01:35:24Z         |-23.4130405        |-46.750367         |
|2023-09-09 22:32|848L-10          |2               |33576                     |15208         |2023-09-10T01:31:38Z         |-23.4130405        |-46.750367         |
|202

In [48]:
# Método de obter operação
df_operacao =df.select(df.hr, df.data_extracao, explode('l').alias('DADOS_LINHA')) \
    .withColumnRenamed('hr', 'HORA_API') \
    .select('HORA_API',
            'DATA_EXTRACAO',
            col('DADOS_LINHA.c').alias('LETREIRO_COMPLETO'),
            col('DADOS_LINHA.cl').alias('CODIGO_IDENTIFICADOR'), 
            col('DADOS_LINHA.lt0').alias('LETREIRO_ORIGEM'),
            col('DADOS_LINHA.lt1').alias('LETREIRO_DESTINO'), 
            col('DADOS_LINHA.qv').alias('QTDE_VEICULOS_OPERACAO'),
              )
#     .where('c == "7005-51"') \

df_operacao.show(truncate=False)

+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|HORA_API|DATA_EXTRACAO   |LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM      |LETREIRO_DESTINO   |QTDE_VEICULOS_OPERACAO|
+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|11:36   |2023-09-09 11:36|6069-10          |1154                |TERM. STO. AMARO     |JD. SÃO BERNARDO   |4                     |
|11:36   |2023-09-09 11:36|6L10-41          |34773               |TERM. VARGINHA       |MESSIÂNICA         |4                     |
|11:36   |2023-09-09 11:36|2734-10          |34528               |METRÔ ITAQUERA       |JD. CAMPOS         |3                     |
|11:36   |2023-09-09 11:36|701A-10          |33457               |METRÔ VL. MADALENA   |PQ. EDU CHAVES     |8                     |
|11:36   |2023-09-09 11:36|273R-10          |32994               |METRÔ ARTU

In [43]:
df_dados_completos_linha = spark.read.options(header='True') \
.csv('data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv')

In [44]:
df_dados_completos_linha.show()

23/09/10 18:59:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv
+---+---+-------+-----------+--------------------+--------------------+
|_c0|_c1|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|
+---+---+-------+-----------+--------------------+--------------------+
|  0|  0|975A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|  1|  1|9784-10|          1|CONSÓRCIO TRANSNO...|SPENCER TRANSPORT...|
|  2|  2|9785-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  3|  3|9785-51|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  4|  4|9789-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  5|  5|978A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO GATO PRETO...|

In [49]:
df_dados_completos_linha.join(df_operacao, df_dados_completos_linha.LINHA == df_operacao.LETREIRO_COMPLETO, 'inner') \
.show(truncate=False)

23/09/10 19:00:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv
+----+---+-------+-----------+---------------------------------------+---------------------------------------+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|_c0 |_c1|LINHA  |CODIGO_AREA|CONSOCIO                               |EMPRESA                                |HORA_API|DATA_EXTRACAO   |LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM      |LETREIRO_DESTINO   |QTDE_VEICULOS_OPERACAO|
+----+---+-------+-----------+---------------------------------------+---------------------------------------+--------+----------------+-----------------+-----------